In [2]:
import pandas as pd
import numpy as np

* Load merged preproceesed data

In [9]:
#pd.set_option('display.max_colwidth', None)
split_df = pd.read_csv('rob2n30_data.csv')

* using emotion.txt lixicon after enhancement

In [10]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
import nltk

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Function to preprocess text with negation handling and lemmatization
def preprocess_text_with_negation_and_lemmatization(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    negative_stop_words = set(['not'])
    # Remove the negative stop words from the stop words set
    final_stop_words = stop_words - negative_stop_words
    tokens = [word for word in tokens if word not in final_stop_words]
    
    # Handle negations
    negation_words = set(['not', 'no', 'never', 'none'])
    negated = False
    processed_tokens = []
    
    for token in tokens:
        if token in negation_words:
            negated = True
            continue
        
        lemma = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        
        if negated:
            processed_tokens.append('not_' + lemma)
            negated = False
        else:
            processed_tokens.append(lemma)
    
    return ' '.join(processed_tokens)
split_df['review/text'] = split_df['review/text'].apply(preprocess_text_with_negation_and_lemmatization)

# Load the expanded emotion lexicon
expanded_lexicon = {}
with open('expanded_emotions.txt') as f:
    for line in f:
        word, emotion = line.strip().split(': ')
        expanded_lexicon[word.strip("'")] = emotion.strip("',")

# Create a dictionary to map negated emotions to their corresponding positive emotions
negated_emotion_mapping = {
    'not_happy': 'sad',
    'not_sad': 'happy',
    'not_angry': 'calm',
    'not_fearful': 'brave',
    'not_bored': 'interested',
    'not_attracted': 'uninterested',
    'not_surprised': 'unmoved',
    'not_loved': 'unloved',
    'not_cheated': 'trusted',
    'not_singled_out': 'included'
}

# Function to classify text based on the expanded lexicon
def classify_emotions_with_negation(text):
    emotions = []
    for word in text.split():
        if word in expanded_lexicon:
            emotions.append(expanded_lexicon[word])
        elif word.startswith('not_'):
            base_word = word[4:]
            if base_word in expanded_lexicon:
                original_emotion = expanded_lexicon[base_word]
                negated_emotion = f"not_{original_emotion}"
                if negated_emotion in negated_emotion_mapping:
                    emotions.append(negated_emotion_mapping[negated_emotion])
                else:
                    emotions.append(negated_emotion)
    return emotions

# Sample sentences for testing
sample_sentences = [
    "I enjoyed this book",
    "I am not happy",
    "I am strain",
    "I am not strain",
    "I never enjoyed this book",
]

# Test the updated functions
for sentence in sample_sentences:
    processed = preprocess_text_with_negation_and_lemmatization(sentence)
    emotions = classify_emotions_with_negation(processed)
    print(f"Sentence: {sentence}")
    print(f"Processed: {processed}")
    print(f"Emotions: {emotions}\n")


# # Apply the classifier to reviews
split_df['emotions.txt'] = split_df['review/text'].apply(classify_emotions_with_negation)
# Count the emotions in each row
split_df['emotion_counts'] = split_df['emotions.txt'].apply(Counter)
# # Aggregate emotions for each book

Sentence: I enjoyed this book
Processed: enjoy book
Emotions: ['attached']

Sentence: I am not happy
Processed: not_happy
Emotions: ['sad']

Sentence: I am strain
Processed: strain
Emotions: ['anxious']

Sentence: I am not strain
Processed: not_strain
Emotions: ['not_anxious']

Sentence: I never enjoyed this book
Processed: not_enjoy book
Emotions: ['not_attached']



In [4]:
data.head()

,Id,Title,profileName,review/helpfulness,review/score,review/summary,review/text,description,authors,image,publisher,publishedDate,categories,sentiment,sentiment_scores,emotions.txt,emotion_counts
0,0826414346,Dr. Seuss: American Icon,Kevin Killian,10/10,5.0,Really Enjoyed It,care much dr seuss reading philip nel book cha...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_positive,0.570646,"['fearful', 'fearless', 'adequate', 'fearful',...","Counter({'fearful': 4, 'attached': 4, 'happy':..."
1,0826414346,Dr. Seuss: American Icon,John Granger,10/11,5.0,Essential for every personal and Public Library,people become book read child father man dr se...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_positive,0.553887,"['attached', 'happy', 'focused', 'attached', '...","Counter({'attached': 7, 'focused': 4, 'cheated..."
2,0826414346,Dr. Seuss: American Icon,"Roy E. Perry ""amateur philosopher""",7/7,4.0,Phlip Nel gives silly Seuss a serious treatment,theodore seuss geisel 1904 1991 aka quot dr se...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_neutral,0.712725,"['adequate', 'happy', 'attached', 'attached', ...","Counter({'attached': 8, 'happy': 6, 'cheated':..."
3,0826414346,Dr. Seuss: American Icon,"D. H. Richards ""ninthwavestore""",3/3,4.0,Good academic overview,philip nel dr seuss american iconthis basicall...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_positive,0.582963,"['cheated', 'attracted', 'happy', 'fearless', ...","Counter({'attracted': 5, 'attached': 5, 'cheat..."
4,0826414346,Dr. Seuss: American Icon,Malvin,2/2,4.0,One of America's greatest creative talents,dr seuss american icon philip nel thoughtful d...,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_neutral,0.598348,"['cheated', 'singled out', 'focused', 'focused...","Counter({'focused': 5, 'cheated': 3, 'happy': ..."


In [11]:
split_df.to_csv('rob2n30_data.csv', index=False)

In [4]:
merged_df=pd.read_csv('3n160.csv')

In [4]:
# Select the desired data (records from 10000 to 100000)
split_df = split_df.iloc[149999:160000]  # Python indexing starts from 0

  # Save the split data to a new CSV file
split_df.to_csv('3n150.csv', index=False)  # Avoid saving index column

print(f"Data split from record 100000 to 150000 and saved ")

Data split from record 100000 to 150000 and saved 


In [5]:
split_df.head()

,Id,Title,profileName,review/helpfulness,review/score,review/summary,review/text,description,authors,image,publisher,publishedDate,categories
19999,0764538926,Betty Crocker 4-Ingredient Dinners,Penmouse,2/2,5.0,Easy and tasty recipes,made several recipe 4 ingredient dinner cookbo...,"Presents a delicious array of wholesome, easy-...",['Betty Crocker'],http://books.google.com/books/content?id=C4IGA...,Betty Crocker,NaN,['Cooking']
20000,0764538926,Betty Crocker 4-Ingredient Dinners,NaN,8/11,5.0,Cooking is no longer a chore for me.,u gifted culinary department book gift cooking...,"Presents a delicious array of wholesome, easy-...",['Betty Crocker'],http://books.google.com/books/content?id=C4IGA...,Betty Crocker,NaN,['Cooking']
20001,0764538926,Betty Crocker 4-Ingredient Dinners,"Carrie Lynn Plastow ""bluestargirl""",1/1,5.0,Nice!,really like cookbook not overwhelming already ...,"Presents a delicious array of wholesome, easy-...",['Betty Crocker'],http://books.google.com/books/content?id=C4IGA...,Betty Crocker,NaN,['Cooking']
20002,0764538926,Betty Crocker 4-Ingredient Dinners,"Jeffrey C. Brindley ""TAZ""",1/1,5.0,Easy and Tasty,recipe book easy tasty book written clearly re...,"Presents a delicious array of wholesome, easy-...",['Betty Crocker'],http://books.google.com/books/content?id=C4IGA...,Betty Crocker,NaN,['Cooking']
20003,B000GG4J6Y,My Mother's Garden,B. K. Earl,7/7,5.0,Anthology of Multiple Authors' Heartwarming Es...,collection essay authored impressive group wri...,A selection of thirty-seven articles and essay...,['Alice Walker'],http://books.google.com/books/content?id=u7FbA...,Harcourt,NaN,['Social Science']


In [26]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


C:\Users\HP\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
split_df.to_csv('2n100000', index=False)  # Avoid saving index column


In [ ]:
import torch
# Load the pre-trained model and tokenizer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def get_sentiment(text):
    # Check if text is empty
    if not text:
        return 'roberta_neutral', 0.0
    
    # Split the text into chunks of 512 tokens
    max_length = 512
    texts = [text[i:i+512] for i in range(0, len(text), 512)]
    
    scores_sum = {
        'roberta_negative': 0,
        'roberta_neutral': 0,
        'roberta_positive': 0
    }
    
    for txt in texts:
        encoded_text = tokenizer(txt, return_tensors='pt')
        with torch.no_grad():
            output = model(**encoded_text)
        scores = output.logits[0].detach().numpy()  # Assuming output.logits returns the scores
        scores = np.exp(scores) / np.exp(scores).sum()  # Apply softmax
        
        scores_dict = {
            'roberta_negative': scores[0],
            'roberta_neutral': scores[1],
            'roberta_positive': scores[2]
        }
        
        # Sum up the scores
        for key in scores_sum.keys():
            scores_sum[key] += scores_dict[key]
    
    # Compute the average scores
    scores_avg = {key: val / len(texts) for key, val in scores_sum.items()}
    
    # Get the sentiment with the maximum average score
    overall_sentiment = max(scores_avg, key=scores_avg.get)
    
    return overall_sentiment, scores_avg[overall_sentiment]

# Apply the function to the text column
merged_df['sentiment'], merged_df['sentiment_scores'] = zip(*merged_df['review/text'].apply(get_sentiment))
merged_df.to_csv('rob3n160.csv', index=False)

In [8]:
split_df.to_csv('rob3n150.csv', index=False)
split_df.head()

,Id,Title,profileName,review/helpfulness,review/score,review/summary,review/text,description,authors,image,publisher,publishedDate,categories,sentiment,sentiment_scores,emotions.txt,emotion_counts
99999,1580627560,Why Men Love Bitches: From Doormat to Dreamgir...,chris,4/4,3.0,"Worth reading, with a few caveats",book try address subject relate men seem take ...,Describes why men are attracted to strong wome...,['Sherry Argov'],http://books.google.com/books/content?id=n2aZD...,Simon and Schuster,NaN,['Family & Relationships'],roberta_neutral,0.563655,"[singled out, adequate, attached, loved, hated...","{'singled out': 2, 'adequate': 1, 'attached': ..."
100000,1580627560,Why Men Love Bitches: From Doormat to Dreamgir...,"Vee ""Not a Doormat anymore""",4/4,5.0,So True Til It Hurt,reading book realize many mistake make previou...,Describes why men are attracted to strong wome...,['Sherry Argov'],http://books.google.com/books/content?id=n2aZD...,Simon and Schuster,NaN,['Family & Relationships'],roberta_neutral,0.495437,"[attached, derailed, attracted, lost, attached...","{'attached': 4, 'derailed': 1, 'attracted': 3,..."
100001,1580627560,Why Men Love Bitches: From Doormat to Dreamgir...,"R. Clarke ""essence01""",4/4,5.0,On Point!,book point know talk different relationship co...,Describes why men are attracted to strong wome...,['Sherry Argov'],http://books.google.com/books/content?id=n2aZD...,Simon and Schuster,NaN,['Family & Relationships'],roberta_positive,0.628836,"[attached, focused, esteemed, esteemed, belitt...","{'attached': 1, 'focused': 2, 'esteemed': 2, '..."
100002,1580627560,Why Men Love Bitches: From Doormat to Dreamgir...,NaN,4/4,5.0,The Best Book Ever!!!,book initially seem like book told men really ...,Describes why men are attracted to strong wome...,['Sherry Argov'],http://books.google.com/books/content?id=n2aZD...,Simon and Schuster,NaN,['Family & Relationships'],roberta_neutral,0.615623,"[attached, attached, attached, fearless, belit...","{'attached': 6, 'fearless': 1, 'belittled': 1,..."
100003,1580627560,Why Men Love Bitches: From Doormat to Dreamgir...,Vanessa Lopez,16/21,3.0,Good for Doormats but not good for the real thing,book good doormat honestly though page 95 book...,Describes why men are attracted to strong wome...,['Sherry Argov'],http://books.google.com/books/content?id=n2aZD...,Simon and Schuster,NaN,['Family & Relationships'],roberta_neutral,0.575698,"[fearless, sad, entitled, sad, sad, sad, happy...","{'fearless': 1, 'sad': 5, 'entitled': 1, 'happ..."


In [12]:
import os

chunks_folder = 'C:/Users/HP/Downloads/archive/bookrec_emotions'
# List all CSV files in the directory
chunk_files = [os.path.join(chunks_folder, f) for f in os.listdir(chunks_folder) if f.endswith('.csv')]

# Read and concatenate all chunks
all_chunks = [pd.read_csv(chunk) for chunk in chunk_files]
combined_df = pd.concat(all_chunks, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('C:/Users/HP/Downloads/archive/combined_file.csv', index=False)


In [13]:
books=pd.read_csv('filtered_book.csv')
merged_df=pd.read_csv('combined_file.csv')

In [22]:
merged_df=pd.read_csv('combined_file.csv')

In [29]:
merged_df.head()

,Id,Title,profileName,review/helpfulness,review/score,review/summary,review/text,description,authors,image,publisher,publishedDate,categories,sentiment,sentiment_scores,emotions.txt
0,0826414346,Dr. Seuss: American Icon,Kevin Killian,10/10,5.0,Really Enjoyed It,care much dr seuss reading philip nel book change mind good testimonial power rel write think rel play dr seuss ultimate compliment treat serious poet well one 20th century interest visual artist reading book decide trip mandeville collection library university california san diego order could visit incredible seuss geisel hold almost much take like william butler yeats seuss lead career constantly shift metamoprhized meet new historical political cirsumstances seem leftist conservative different juncture career politics art nel show u cartoonist fabled pm magazine like andy warhol serve time slave ad business service amuse broadening mind u child nel hesitate administer sound spank seuss industry since death see fit license kind awful product include recent cat hat film mike myers oh cat astrophe book great especially recommend work picture editor give u bounty good illustration,"Philip Nel takes a fascinating look into the key aspects of Seuss's career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seuss's famously loopy artistic style - what Nel terms an ""energetic cartoon surrealism"" - has been equally important, inspiring artists like filmmaker Tim Burton and illustrator Lane Smith. --from back cover",['Philip Nel'],http://books.google.com/books/content?id=IjvHQsCn_pgC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_positive,0.570646,"['fearful', 'fearless', 'adequate', 'fearful', 'focused', 'singled out', 'esteemed', 'loved', 'attached', 'sad', 'fearful', 'attached', 'happy', 'anxious', 'attached', 'happy', 'fearful', 'attached', 'hated', 'happy', 'cheated', 'focused', 'fearless']"
1,0826414346,Dr. Seuss: American Icon,John Granger,10/11,5.0,Essential for every personal and Public Library,people become book read child father man dr seuss theodor seuss geisel influential author poet artist modern time daddy large family learn read dr seuss memorize many book via repeat reading young child prof nel brilliant american icon long await treat last serious treatment remarkable genius engage read fill remarkable insight especially enjoy learn care admit prof nel discussion disneyfication seuss nel link fail american copyright law side dr seuss side new political genesis secular morality wwii cartoon work pm magazine chapter geisel poetry artwork link nel make seuss historical avant guarde alone make book must buy parent serious reader not_mention public library reader nel book find engage write style make book fun read impart mountain information important idea simply best comprehensive book yet write work seuss geisel certainly standard many year come thank prof nel wherever reader grow good doctor grow year later book write encyclopeadic knowledge child literature medium genre scan verse cubist painting explains power limit popularity seuss phenomenon,"Philip Nel takes a fascinating look into the key aspects of Seuss's career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seuss's famously loopy artistic style - what Nel terms an ""energetic cartoon surrealism"" - has been equally important, inspiring artists like filmmaker Tim Burton and illustrator Lane Smith. --from back cover",['Philip Nel'],http:

In [33]:
from collections import Counter

# Define a function to get the top three emotions
def get_top_emotions(emotion_list):
    # Convert the string representation of list to an actual list
    emotion_list = eval(emotion_list)
    # Count the occurrences of each emotion
    emotion_counts = Counter(emotion_list)
    # Get the top three emotions
    top_three = [emotion for emotion, count in emotion_counts.most_common(3)]
    return top_three

# Apply the function to create the 'top_emotions' column
merged_df['top_emotions'] = merged_df['emotions.txt'].apply(get_top_emotions)
print(merged_df.head())

           Id                     Title                         profileName  \
0  0826414346  Dr. Seuss: American Icon                       Kevin Killian   
1  0826414346  Dr. Seuss: American Icon                        John Granger   
2  0826414346  Dr. Seuss: American Icon  Roy E. Perry "amateur philosopher"   
3  0826414346  Dr. Seuss: American Icon     D. H. Richards "ninthwavestore"   
4  0826414346  Dr. Seuss: American Icon                              Malvin   

  review/helpfulness  review/score  \
0              10/10           5.0   
1              10/11           5.0   
2                7/7           4.0   
3                3/3           4.0   
4                2/2           4.0   

                                    review/summary  \
0                                Really Enjoyed It   
1  Essential for every personal and Public Library   
2  Phlip Nel gives silly Seuss a serious treatment   
3                           Good academic overview   
4       One of America's g

In [28]:
merged_df=merged_df.drop(columns=['emotion_counts'])

In [34]:
# Aggregate emotion counts for each book
def aggregate_emotions(emotion_list):
    total_counter = Counter()
    for emotion_counter in emotion_list:
        total_counter.update(emotion_counter)
    return total_counter

# Group by 'tilte' and aggregate the emotion counts
book_emotion_counts = merged_df.groupby('Title')['top_emotions'].apply(list).apply(aggregate_emotions)

# Convert back to a DataFrame
book_emotion_counts_df = book_emotion_counts.reset_index()

# Display first few rows of the book emotion counts DataFrame
print(book_emotion_counts_df.head(20))

                                                                                                                  Title  \
0                                                          "Boots and saddles" ;: Or Life in Dakota with General Custer   
1                                              "Catch 'em alive Jack";: The life and adventures of an American pioneer,   
2   "Doers of the Word": African-American Women Speakers and Writers in the North (1830-1880) (Race & American Culture)   
3                                                                                                  "Our Brown-Eyed Boy"   
4                                   'Pataphysics: The Poetics of an Imaginary Science (Avant-Garde & Modernism Studies)   
5                                                                            'night, Mother: A Play (Mermaid Dramabook)   
6   (ESV) English Standard Version Large Print Bible. Premium Bonded Leather, Black, Red Letter Text (English Language)   
7               

In [35]:
# Extract top emotions for each book
def get_top_emotions(emotion_counter, top_n=3):
    return [emotion for emotion, count in emotion_counter.most_common(top_n)]

book_emotion_counts_df['top_emotions'] = book_emotion_counts_df['top_emotions'].apply(lambda x: get_top_emotions(x, top_n=3))

# Display first few rows of the DataFrame with top emotions
print(book_emotion_counts_df.head())

                                                                                                                 Title  \
0                                                         "Boots and saddles" ;: Or Life in Dakota with General Custer   
1                                             "Catch 'em alive Jack";: The life and adventures of an American pioneer,   
2  "Doers of the Word": African-American Women Speakers and Writers in the North (1830-1880) (Race & American Culture)   
3                                                                                                 "Our Brown-Eyed Boy"   
4                                  'Pataphysics: The Poetics of an Imaginary Science (Avant-Garde & Modernism Studies)   

                     top_emotions  
0     [happy, adequate, attached]  
1         [adequate, free, alone]  
2                [attracted, sad]  
3    [attached, happy, attracted]  
4  [attracted, adequate, focused]  


In [24]:
pd.set_option('display.max_colwidth', None)

In [17]:
book_emotion_counts_df.head(20)

,Title,emotion_counts,top_emotions
0,"""Boots and saddles"" ;: Or Life in Dakota with ...","{'[': 6, ''': 134, 'f': 15, 'o': 11, 'c': 18, ...","[', a, e]"
1,"""Catch 'em alive Jack"";: The life and adventur...","{'[': 2, ''': 40, 'a': 24, 't': 21, 'c': 6, 'h...","[', e, a]"
2,"""Doers of the Word"": African-American Women Sp...","{'[': 1, ''': 4, 'a': 3, 't': 3, 'r': 1, 'c': ...","[', a, t]"
3,"""Our Brown-Eyed Boy""","{'[': 17, ''': 464, 'f': 38, 'o': 55, 'c': 106...","[', e, a]"
4,'Pataphysics: The Poetics of an Imaginary Scie...,"{'[': 1, ''': 138, 'a': 66, 'd': 50, 'e': 88, ...","[', e, t]"
5,"'night, Mother: A Play (Mermaid Dramabook)","{'[': 20, ''': 730, 'n': 54, 'o': 93, 't': 311...","[', e, a]"
6,(ESV) English Standard Version Large Print Bib...,"{'[': 438, ''': 11282, 'a': 5319, 't': 4986, '...","[', e, ]"
7,*OP Dharma Book: Devil Tigers (Kindred of the ...,"{'[': 3, ''': 74, 'f': 2, 'e': 27, 'a': 45, 'r...","[', a, ,]"
8,.NET Multithreading,"{'[': 9, ''': 220, 'a': 117, 't': 115, 'c': 47...","[', e, a]"
9,"1,999 Facts About Blacks: A Sourcebook of Afri...","{'[': 4, ''': 48, 'h': 8, 'a': 23, 'p': 7, 'y'...","[', a, ,]"


In [36]:
unique_titles = merged_df['Title'].unique()

# Display the first few unique titles to verify
print(unique_titles[:10])

['Dr. Seuss: American Icon' 'Wonderful Worship in Smaller Churches'
 'Whispers of the Wicked Saints'
 'The Church of Christ: A Biblical Ecclesiology for Today'
 'Saint Hyacinth of Poland'
 "Rising Sons and Daughters: Life Among Japan's New Young"
 "Muslim Women's Choices: Religious Belief and Social Reality (Cross Cultural Perspectives on Women)"
 'Dramatica for Screenwriters'
 'Mensa Number Puzzles (Mensa Word Games for Kids)'
 'Vector Quantization and Signal Compression (The Springer International Series in Engineering and Computer Science)']


In [34]:
# Filter the original books DataFrame to create a smaller dataset with only the relevant titles
filtered_books_df = books[books['Title'].isin(unique_titles)]

# Display the first few rows of the filtered books DataFrame
print(filtered_books_df.head())

# Optional: Save the filtered DataFrame to a new CSV file
filtered_books_df.to_csv('10recbooks.csv', index=False)


                                                     Title  \
0                                 Dr. Seuss: American Icon   
1                    Wonderful Worship in Smaller Churches   
2                            Whispers of the Wicked Saints   
3  The Church of Christ: A Biblical Ecclesiology for Today   
4                                 Saint Hyacinth of Poland   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [39]:
filtered_books_df = books[books['Title'].isin(unique_titles)]

# Merge top emotions into books_df
books_with_emotions_df = pd.merge(filtered_books_df, book_emotion_counts_df[['Title', 'top_emotions']], on='Title', how='left')

# Display first few rows of the updated books DataFrame
print(books_with_emotions_df.head())

# Optional: Save the updated DataFrame to a new CSV file
books_with_emotions_df.to_csv('10recbooks.csv', index=False)


                                                     Title  \
0                                 Dr. Seuss: American Icon   
1                    Wonderful Worship in Smaller Churches   
2                            Whispers of the Wicked Saints   
3  The Church of Christ: A Biblical Ecclesiology for Today   
4                                 Saint Hyacinth of Poland   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [40]:
# Group by 'booktitle' and calculate the number of reviews and the average rating
reviews_summary = merged_df.groupby('Title').agg(
    num_reviews=pd.NamedAgg(column='review/text', aggfunc='count'),
    avg_rating=pd.NamedAgg(column='review/score', aggfunc=lambda x: round(x.mean(), 1))
).reset_index()
#reviews_summary['avg_rating'] = reviews_summary['avg_rating'].astype(int)

# Display the summary DataFrame
print(reviews_summary.head())

                                                                                                                 Title  \
0                                                         "Boots and saddles" ;: Or Life in Dakota with General Custer   
1                                             "Catch 'em alive Jack";: The life and adventures of an American pioneer,   
2  "Doers of the Word": African-American Women Speakers and Writers in the North (1830-1880) (Race & American Culture)   
3                                                                                                 "Our Brown-Eyed Boy"   
4                                  'Pataphysics: The Poetics of an Imaginary Science (Avant-Garde & Modernism Studies)   

   num_reviews  avg_rating  
0            6         3.8  
1            2         4.5  
2            1         3.0  
3           17         3.4  
4            1         1.0  


In [42]:
# Merge the summary DataFrame with the books DataFrame
books_with_reviews_df = pd.merge(books_with_emotions_df, reviews_summary, on='Title', how='left')

# Display the updated books DataFrame
print(books_with_reviews_df.head(20))

# Optional: Save the updated DataFrame to a new CSV file
books_with_reviews_df.to_csv('150Recbooks.csv', index=False)


                                                                                                                                          Title  \
0                                                                                                                      Dr. Seuss: American Icon   
1                                                                                                         Wonderful Worship in Smaller Churches   
2                                                                                                                 Whispers of the Wicked Saints   
3                                                                                       The Church of Christ: A Biblical Ecclesiology for Today   
4                                                                                                                      Saint Hyacinth of Poland   
5                                                                                       Rising Sons and Daughters: Lif

In [51]:
books_with_reviews_df.head(20)

,Title,description,authors,image,publisher,publishedDate,categories,top_emotions,num_reviews,avg_rating
0,Dr. Seuss: American Icon,"Philip Nel takes a fascinating look into the key aspects of Seuss's career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seuss's famously loopy artistic style - what Nel terms an ""energetic cartoon surrealism"" - has been equally important, inspiring artists like filmmaker Tim Burton and illustrator Lane Smith. --from back cover",['Philip Nel'],http://books.google.com/books/content?id=IjvHQsCn_pgC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api,A&C Black,2005-01-01,['Biography & Autobiography'],"[attached, cheated, happy]",9,4.6
1,Wonderful Worship in Smaller Churches,"This resource includes twelve principles in understanding small church worship, fifteen practices for planning worship with fewer than 100 people, and suggestions for congregational study.",['David R. Ray'],http://books.google.com/books/content?id=2tsDAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,NaN,NaN,['Religion'],"[belittled, happy, esteemed]",4,5.0
2,Whispers of the Wicked Saints,"Julia Thomas finds her life spinning out of control after the death of her husband, Richard. Julia turns to her minister for comfort when she finds herself falling for him with a passion that is forbidden by the church. Heath Sparks is a man of God who is busy taking care of his quadriplegic wife who was seriously injured in a sever car accident. In an innocent effort to reach out to a lonely member of his church, Heath finds himself as the man and not the minister as Heath and Julia surrender their bodies to each other and face the wrath of God. Julia finds herself in over her head as she faces a deadly disease, the loss of her home and whispers about her wicked affair. Julia leaves the states offering her body as a living sacrifice in hopes of finding a cure while her heart remains thousands of miles away hoping to one day reunite with the man who holds it hostage.Whispers of the Wicked Saints is a once in a lifetime romance that is breath taking, defying all the rules of romance and bending the laws of love.",['Veronica Haddon'],http://books.google.com/books/content?id=aRSIgJlq6JwC&printsec=frontcover&img=1&zoom=1&source=gbs_api,iUniverse,NaN,['Fiction'],"[attached, sad, adequate]",32,3.7
3,The Church of Christ: A Biblical Ecclesiology for Today,"In The Church of Christ: A Biblical Ecclesiology for Today, respected biblical scholar Everett Ferguson presents a genuine biblical theology of the church. By systematically examining the New Testament's teaching on the existence, meaning, and purpose of the church, providing responsible coverage of the traditional topics in ecclesiology, and carefully grounding ecclesiology in the person and work of Christ, Ferguson unveils a comprehensive model of the church that is both biblically centered and relevant to a world on the verge of the twenty-first century.",['Everett Ferguson'],http://books.google.com/books/content?id=kVqRaiPlx88C&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api,Wm. B. Eerdmans Publishing,NaN,['Religion'],"[focused, adequate, attracted]",4,4.5
4,Saint Hyacinth of Poland,"The story for children 10 and up of St. Hyacinth, the Dominican who planted the Faith in Poland, Lithuania and Russia and worked many miracles. He went to Rome, where he met St. Dominic, and was one of the first to receive at his hands the habit of the newly established Order of Friars Preachers. After his novitiate he made his religious profession, and was made superior of the little band of missionaries sent to Poland to preach. Impr. 189 pgs 16 Illus, PB",['Mary Fabyan Windeatt'],http://books.google.com/books/content?id=lmLqAAAACAAJ&prin

In [36]:
merged_df

,Id,Title,profileName,review/helpfulness,review/score,review/summary,review/text,description,authors,image,publisher,publishedDate,categories,sentiment,sentiment_scores,emotions.txt,emotion_counts
0,0826414346,Dr. Seuss: American Icon,Kevin Killian,10/10,5.0,Really Enjoyed It,care much dr seuss reading philip nel book change mind good testimonial power rel write think rel play dr seuss ultimate compliment treat serious poet well one 20th century interest visual artist reading book decide trip mandeville collection library university california san diego order could visit incredible seuss geisel hold almost much take like william butler yeats seuss lead career constantly shift metamoprhized meet new historical political cirsumstances seem leftist conservative different juncture career politics art nel show u cartoonist fabled pm magazine like andy warhol serve time slave ad business service amuse broadening mind u child nel hesitate administer sound spank seuss industry since death see fit license kind awful product include recent cat hat film mike myers oh cat astrophe book great especially recommend work picture editor give u bounty good illustration,"Philip Nel takes a fascinating look into the key aspects of Seuss's career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seuss's famously loopy artistic style - what Nel terms an ""energetic cartoon surrealism"" - has been equally important, inspiring artists like filmmaker Tim Burton and illustrator Lane Smith. --from back cover",['Philip Nel'],http://books.google.com/books/content?id=IjvHQsCn_pgC&printsec=frontcover&img=1&zoom=1&edge=curl&source=gbs_api,A&C Black,2005-01-01,['Biography & Autobiography'],roberta_positive,0.570646,"[fearful, fearless, adequate, fearful, focused, singled out, esteemed, loved, attached, sad, fearful, attached, happy, anxious, attached, happy, fearful, attached, hated, happy, cheated, focused, fearless]","{'fearful': 4, 'fearless': 2, 'adequate': 1, 'focused': 2, 'singled out': 1, 'esteemed': 1, 'loved': 1, 'attached': 4, 'sad': 1, 'happy': 3, 'anxious': 1, 'hated': 1, 'cheated': 1}"
1,0826414346,Dr. Seuss: American Icon,John Granger,10/11,5.0,Essential for every personal and Public Library,people become book read child father man dr seuss theodor seuss geisel influential author poet artist modern time daddy large family learn read dr seuss memorize many book via repeat reading young child prof nel brilliant american icon long await treat last serious treatment remarkable genius engage read fill remarkable insight especially enjoy learn care admit prof nel discussion disneyfication seuss nel link fail american copyright law side dr seuss side new political genesis secular morality wwii cartoon work pm magazine chapter geisel poetry artwork link nel make seuss historical avant guarde alone make book must buy parent serious reader not_mention public library reader nel book find engage write style make book fun read impart mountain information important idea simply best comprehensive book yet write work seuss geisel certainly standard many year come thank prof nel wherever reader grow good doctor grow year later book write encyclopeadic knowledge child literature medium genre scan verse cubist painting explains power limit popularity seuss phenomenon,"Philip Nel takes a fascinating look into the key aspects of Seuss's career - his poetry, politics, art, marketing, and place in the popular imagination."" ""Nel argues convincingly that Dr. Seuss is one of the most influential poets in America. His nonsense verse, like that of Lewis Carroll and Edward Lear, has changed language itself, giving us new words like ""nerd."" And Seuss's famously loopy artistic style - what Nel terms an ""energetic cartoon su

# didnot used 

In [ ]:
def recommend_books(row, books_df):
    review_stars = row['review/score']
    review_emotions = row['review_emotions']
    book_emotions = row['emotion_counts']  # Assuming this column exists in the dataset
    threshold = 0.5  # Define your threshold for emotion matching

    if review_stars < 3:
        # Check if review content emotions match book emotions above the threshold
        match_count = len(set(review_emotions) & set(book_emotions))
        if match_count / len(book_emotions) > threshold:
            # Recommend a book which differs from the rated book
            recommended_books = books_df[books_df['Title'] != row['Title']]
        else:
            # Recommend a book similar to the rated one
            recommended_books = books_df[books_df['Title'] == row['Title']]
    elif 3 <= review_stars < 5:
        # Recommend a better book similar to the rated one
        recommended_books = books_df[books_df['Title'] == row['Title']]
    elif review_stars == 5:
        # Recommend a book similar to the rated one
        recommended_books = books_df[books_df['Title'] == row['Title']]
    
    # Return the recommended books (could be a subset or a single book)
    return recommended_books.head(1)  # Example: returning the first recommendation


# Apply the recommendation function to each row
#df['recommendations'] = df.apply(lambda row: recommend_books(row, df), axis=1)


# Recommendation steps

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

books_with_reviews_df['top_emotions'] = books_with_reviews_df['top_emotions'].apply(lambda x: ' '.join(x))

In [55]:
# Vectorize the emotions
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books_with_reviews_df['top_emotions'])

# Convert TF-IDF matrix to a DataFrame for better visualization
emotion_features = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

In [57]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the ratings
scaler = MinMaxScaler()
books_with_reviews_df['normalized_rating'] = scaler.fit_transform(books_with_reviews_df[['avg_rating']])


In [58]:
# Combine emotion features and normalized ratings
combined_features = pd.concat([emotion_features, books_with_reviews_df['normalized_rating']], axis=1)

# Display the combined feature vectors
print(combined_features)

     adequate     angry   anxious  apathetic  attached  attracted  average  \
0    0.000000  0.000000  0.000000        0.0  0.303609   0.000000      0.0   
1    0.000000  0.000000  0.000000        0.0  0.000000   0.000000      0.0   
2    0.723399  0.000000  0.000000        0.0  0.314761   0.000000      0.0   
3    0.690160  0.000000  0.000000        0.0  0.000000   0.517836      0.0   
4    0.000000  0.893102  0.000000        0.0  0.272670   0.000000      0.0   
..        ...       ...       ...        ...       ...        ...      ...   
599  0.000000  0.000000  0.000000        0.0  0.276943   0.000000      0.0   
600  0.000000  0.000000  0.000000        0.0  0.424269   0.000000      0.0   
601  0.000000  0.000000  0.000000        0.0  0.303609   0.000000      0.0   
602  0.761215  0.000000  0.000000        0.0  0.331215   0.000000      0.0   
603  0.000000  0.000000  0.747087        0.0  0.179976   0.000000      0.0   

     belittled  bored  burdened  ...  not_attached  not_fearles

In [82]:
def recommend_for_new_user(emotion, df, combined_features, top_n=5):
    # Vectorize the input emotion
    emotion_vector = tfidf.transform([emotion]).toarray()

    # Append a 0 for the rating normalization
    emotion_vector = pd.DataFrame(emotion_vector, columns=tfidf.get_feature_names_out())
    emotion_vector['normalized_rating'] = 0

    # Calculate similarity
    similarities = combined_features.apply(lambda x: linear_kernel([x], emotion_vector)[0][0], axis=1)

    # Add similarity scores to the dataframe
    df['similarity'] = similarities

    # Filter and sort by similarity and rating
    recommended_books = df.sort_values(by=['similarity', 'normalized_rating'], ascending=[False, False])

    return recommended_books[['Title', 'normalized_rating', 'top_emotions']].head(top_n)

# Example usage for a new user wanting "exciting" books
new_user_recommendations = recommend_for_new_user("happy", books_with_reviews_df, combined_features)
print("Recommendations for new user based on emotion 'exciting adventurous':")
new_user_recommendations



Recommendations for new user based on emotion 'exciting adventurous':


,Title,normalized_rating,top_emotions
584,"Bermuda '99: The Complete Guide with Secluded Beaches, Resorts and Cottages and Seaside Golf (Fodor's Bermuda)",1.000,happy lost
359,Meaning in Language: An Introduction to Semantics and Pragmatics (Oxford Textbooks in Linguistics),1.000,lost happy attached
515,"Mommy, Why Did Jesus Have to Die?",1.000,happy attached lost
593,"Get Busy, Beaver! (Book and Audio CD) (Paperback)",0.875,lost happy attached
14,Alaska Sourdough,0.825,attached happy lost


In [75]:
specific_book = books_with_reviews_df[books_with_reviews_df['Title'] == "The Tao of I Ching: Way to Divination"]

# Display the filtered DataFrame
specific_book

,Title,description,authors,image,publisher,publishedDate,categories,top_emotions,num_reviews,avg_rating,normalized_rating,similarity
131,The Tao of I Ching: Way to Divination,A new and refreshing way to understand and utilize the I Ching.,['Tsung Hwa Jou'],http://books.google.com/books/content?id=820EAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,Tuttle Publishing,1989-12-15,"['Body, Mind & Spirit']",focused happy,1,4.0,0.75,0.614806


In [80]:
specific_book = merged_df[merged_df['Title'] == "Wonderful Worship in Smaller Churches"]

# Display the filtered DataFrame
specific_book

,Id,Title,profileName,review/helpfulness,review/score,review/summary,review/text,description,authors,image,publisher,publishedDate,categories,sentiment,sentiment_scores,emotions.txt,emotion_counts
9,0829814000,Wonderful Worship in Smaller Churches,Rev. Pamela Tinnin,8/10,5.0,Outstanding Resource for Small Church Pastors,finish book quot wonderful worship small church quot david ray outstanding resource small church pastor book quot big small church book quot reread regularly refer often may reorder quot big small church book quot copy nearly worn two choice resource pastor small church would two hope author considers focus christian education another book section quot big small church book quot christian education concentrate one important church function individually make huge difference proven new book tell colleague rev ray book worth every dollar,"This resource includes twelve principles in understanding small church worship, fifteen practices for planning worship with fewer than 100 people, and suggestions for congregational study.",['David R. Ray'],http://books.google.com/books/content?id=2tsDAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,NaN,NaN,['Religion'],roberta_positive,0.521750,"[happy, esteemed, belittled, happy, belittled, happy, belittled, attracted, happy, belittled, belittled, focused, happy, belittled, focused, esteemed]","{'happy': 5, 'esteemed': 2, 'belittled': 6, 'attracted': 1, 'focused': 2}"
10,0829814000,Wonderful Worship in Smaller Churches,Dr. Terry W. Dorsett,1/1,5.0,Small Churches CAN Have Wonderful Worship,many small church feel like not_great worship lack resource large church take grant ray list thou shalts worth price book description various personality small church humorous true book give encouragement well practical advice,"This resource includes twelve principles in understanding small church worship, fifteen practices for planning worship with fewer than 100 people, and suggestions for congregational study.",['David R. Ray'],http://books.google.com/books/content?id=2tsDAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,NaN,NaN,['Religion'],roberta_neutral,0.647796,"[belittled, attached, sad, esteemed, happy, loved, belittled, happy, focused]","{'belittled': 2, 'attached': 1, 'sad': 1, 'esteemed': 1, 'happy': 2, 'loved': 1, 'focused': 1}"
11,0829814000,Wonderful Worship in Smaller Churches,"Cynthia L. Lajoy ""Cindy La Joy""",1/1,5.0,Not Just for Pastors!,finish reading amaze book come away entirely different perspective small congregation ministry leader easy become depressed not_accomplish overwork everyone small church often fail see accomplish extraordinary small congregation many plus outline book work small day mega church reminder good one good tip alter worship small congregation value unique size work well thousand u find church membership 100 le discourage worship attendence 30 40 summer book uplift perspective help energize unlike many tome nature easy read layperson gobble immediately put use david ray write style accessible make assumption knowledge reader offering explanation need helpful pick even not_pastor gain something guarantee,"This resource includes twelve principles in understanding small church worship, fifteen practices for planning worship with fewer than 100 people, and suggestions for congregational study.",['David R. Ray'],http://books.google.com/books/content?id=2tsDAAAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api,NaN,NaN,['Religion'],roberta_positive,0.703390,"[lost, alone, belittled, happy, sad, cheated, belittled, cheated, attached, belittled, cheated, belittled, fearless, fearless, esteemed, belittled, esteemed, alone, cheated, focused, powerless, esteemed, happy, adequate, happy, attached, focused, adequate, attracted, fearless]","{'lost': 1, 'alone': 2, 'belittled': 5, 'happy': 3, 'sad': 1, 'cheated': 4, 'attached': 2, 'fearless': 3, 'esteemed': 3, 'focused': 2, 'powerless': 1, 'adequate': 2, 'attracted': 1}"
12,0829814000,Wonderful

# For cold Start

In [85]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import linear_kernel

# Sample dataset with list of emotions
books_data = [
    {"title": "Book A", "reviews": "Great book with exciting adventures.", "rating": 4, "emotion": ["exciting", "adventurous"]},
    {"title": "Book B", "reviews": "A sad but enlightening story.", "rating": 5, "emotion": ["sad", "enlightening"]},
    {"title": "Book C", "reviews": "An inspiring tale of triumph.", "rating": 4, "emotion": ["inspiring", "triumphant"]},
    {"title": "Book D", "reviews": "Scary and thrilling.", "rating": 3, "emotion": ["scary", "thrilling"]}
]

# User reading history
user_history = {
    "user1": [{"title": "Book A", "rating": 5}, {"title": "Book C", "rating": 4}],
    "user2": [{"title": "Book B", "rating": 4}, {"title": "Book D", "rating": 3}]
}

# Convert to DataFrame
df_books = pd.DataFrame(books_data)

# Join list of emotions into a single string
df_books['emotion'] = df_books['emotion'].apply(lambda x: ' '.join(x))

# Vectorize the emotions
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_books['emotion'])

# Convert TF-IDF matrix to a DataFrame for better visualization
emotion_features = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Normalize the ratings
scaler = MinMaxScaler()
df_books['normalized_rating'] = scaler.fit_transform(df_books[['rating']])

# Combine emotion features and normalized ratings
combined_features = pd.concat([emotion_features, df_books['normalized_rating']], axis=1)

# Function to create user profiles
def create_user_profile(user_history, df_books, combined_features):
    user_profiles = {}
    
    for user, books in user_history.items():
        user_feature_vector = pd.DataFrame()
        
        for book in books:
            book_title = book['title']
            book_rating = book['rating']
            
            # Get the book features
            book_features = combined_features[df_books['title'] == book_title]
            
            # Scale the book features by the user's rating of the book
            book_features = book_features * book_rating
            
            # Append to user feature vector
            user_feature_vector = pd.concat([user_feature_vector, book_features], axis=0)
        
        # Average the feature vectors to create the user profile
        user_profiles[user] = user_feature_vector.mean(axis=0)
    
    return user_profiles

# Create user profiles
user_profiles = create_user_profile(user_history, df_books, combined_features)
print("User Profiles:")
print(user_profiles)

# Function to recommend books for a user based on their profile and desired emotion
def recommend_books_for_user(user_profile, combined_features, df_books, desired_emotion, top_n=5):
    # Vectorize the desired emotion
    emotion_vector = tfidf.transform([desired_emotion]).toarray()
    
    # Add zeros for the rating normalization to match the dimension
    emotion_vector = pd.DataFrame(emotion_vector, columns=tfidf.get_feature_names_out())
    emotion_vector = pd.concat([emotion_vector, pd.DataFrame([[0]], columns=['normalized_rating'])], axis=1)
    
    # Calculate similarity between user profile and book features
    similarities = combined_features.apply(lambda x: linear_kernel([x], [user_profile])[0][0], axis=1)
    
    # Add similarity scores to the dataframe
    df_books['similarity'] = similarities
    
    # Calculate emotion similarity
    emotion_similarities = combined_features.apply(lambda x: linear_kernel([x], emotion_vector)[0][0], axis=1)
    
    # Add emotion similarity scores to the dataframe
    df_books['emotion_similarity'] = emotion_similarities
    
    # Combine similarity scores with a weight factor (e.g., 0.5 for each)
    df_books['combined_similarity'] = 0.5 * df_books['similarity'] + 0.5 * df_books['emotion_similarity']
    
    # Filter and sort by combined similarity and rating
    recommended_books = df_books.sort_values(by=['combined_similarity', 'rating'], ascending=[False, False])
    
    return recommended_books[['title', 'rating', 'emotion']].head(top_n)

# Example usage for user1 with a desired emotion of "exciting adventurous"
user1_profile = user_profiles['user1']
user1_recommendations = recommend_books_for_user(user1_profile, combined_features, df_books, "exciting adventurous")
print("Recommendations for user1:")
print(user1_recommendations)


User Profiles:
{'user1': adventurous          1.767767
enlightening         0.000000
exciting             1.767767
inspiring            1.414214
sad                  0.000000
scary                0.000000
thrilling            0.000000
triumphant           1.414214
normalized_rating    2.250000
dtype: float64, 'user2': adventurous          0.000000
enlightening         1.414214
exciting             0.000000
inspiring            0.000000
sad                  1.414214
scary                1.060660
thrilling            1.060660
triumphant           0.000000
normalized_rating    2.000000
dtype: float64}
Recommendations for user1:
    title  rating               emotion
0  Book A       4  exciting adventurous
2  Book C       4  inspiring triumphant
1  Book B       5      sad enlightening
3  Book D       3       scary thrilling


In [43]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import linear_kernel

# Sample dataset with list of emotions, author, category, and number of reviews
books_data = [
    {"title": "Book A", "reviews": "Great book with exciting adventures.", "rating": 4, "emotion": ["exciting", "adventurous"], "author": "Author 1", "category": "Adventure", "num_reviews": 10},
    {"title": "Book f", "reviews": "Great book with exciting adventures.", "rating": 5, "emotion": ["exciting", "adventurous"], "author": "Author 1", "category": "Adventure", "num_reviews": 10},
    {"title": "Book B", "reviews": "A sad but enlightening story.", "rating": 5, "emotion": ["sad", "enlightening"], "author": "Author 2", "category": "Drama", "num_reviews": 3},
    {"title": "Book C", "reviews": "An inspiring tale of triumph.", "rating": 4, "emotion": ["inspiring", "triumphant"], "author": "Author 1", "category": "Biography", "num_reviews": 8},
    {"title": "Book D", "reviews": "Scary and thrilling.", "rating": 3, "emotion": ["scary", "thrilling"], "author": "Author 3", "category": "Horror", "num_reviews": 2}
]

# User reading history
user_history = {
    "user1": [{"title": "Book A", "rating": 5}, {"title": "Book C", "rating": 4}],
    "user2": [{"title": "Book B", "rating": 4}, {"title": "Book D", "rating": 3}]
}

# Convert to DataFrame
df_books = pd.DataFrame(books_data)

# Join list of emotions into a single string
df_books['emotion'] = df_books['emotion'].apply(lambda x: ' '.join(x))

# Combine author and category with emotion for feature extraction
df_books['combined_features'] = df_books[['emotion', 'author', 'category']].apply(lambda x: ' '.join(x), axis=1)

# Vectorize the combined features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_books['combined_features'])

# Convert TF-IDF matrix to a DataFrame for better visualization
combined_features = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Normalize the ratings
scaler = MinMaxScaler()
df_books['normalized_rating'] = scaler.fit_transform(df_books[['rating']])

# Add normalized rating to combined features
combined_features['normalized_rating'] = df_books['normalized_rating']

# Function to create user profiles
def create_user_profile(user_history, df_books, combined_features):
    user_profiles = {}
    
    for user, books in user_history.items():
        user_feature_vector = pd.DataFrame()
        
        for book in books:
            book_title = book['title']
            book_rating = book['rating']
            
            # Get the book features
            book_features = combined_features[df_books['title'] == book_title]
            
            # Scale the book features by the user's rating of the book
            book_features = book_features * book_rating
            
            # Append to user feature vector
            user_feature_vector = pd.concat([user_feature_vector, book_features], axis=0)
        
        # Average the feature vectors to create the user profile
        user_profiles[user] = user_feature_vector.mean(axis=0)
    
    return user_profiles

# Create user profiles
user_profiles = create_user_profile(user_history, df_books, combined_features)
print("User Profiles:")
print(user_profiles)

# Function to recommend books for a user based on their profile and desired emotion
def recommend_books_for_user(user_profile, combined_features, df_books, desired_emotion, top_n=5):
    # Combine desired emotion with empty author and category for feature extraction
    desired_combined_features = ' '.join([desired_emotion, '', ''])
    
    # Vectorize the desired features
    desired_vector = tfidf.transform([desired_combined_features]).toarray()
    
    # Add zero for the normalized rating to match the dimension
    desired_vector = pd.DataFrame(desired_vector, columns=tfidf.get_feature_names_out())
    desired_vector['normalized_rating'] = 0
    
    # Calculate similarity between user profile and book features
    similarities = combined_features.apply(lambda x: linear_kernel([x], [user_profile])[0][0], axis=1)
    
    # Add similarity scores to the dataframe
    df_books['similarity'] = similarities
    
    # If the number of reviews is less than 5, set emotion similarity to 0
    df_books['emotion_similarity'] = df_books.apply(
        lambda row: 0 if row['num_reviews'] < 5 else linear_kernel([combined_features.iloc[row.name]], desired_vector)[0][0], axis=1
    )
    
    # Combine similarity scores with a weight factor (e.g., 0.5 for each)
    df_books['combined_similarity'] = 0.5 * df_books['similarity'] + 0.5 * df_books['emotion_similarity']
    
    # Filter and sort by combined similarity and rating
    recommended_books = df_books.sort_values(by=['combined_similarity', 'rating'], ascending=[False, False])
    
    return recommended_books[['title', 'author', 'category', 'rating', 'emotion', 'num_reviews']].head(top_n)

# Example usage for user1 with a desired emotion of "exciting adventurous"
user1_profile = user_profiles['user1']
user1_recommendations = recommend_books_for_user(user1_profile, combined_features, df_books, "exciting adventurous")
print("Recommendations for user1:")
print(user1_recommendations)


User Profiles:
{'user1': adventure            1.366135
adventurous          1.366135
author               1.337372
biography            1.113337
drama                0.000000
enlightening         0.000000
exciting             1.366135
horror               0.000000
inspiring            1.113337
sad                  0.000000
scary                0.000000
thrilling            0.000000
triumphant           1.113337
normalized_rating    2.250000
dtype: float64, 'user2': adventure            0.000000
adventurous          0.000000
author               0.928394
biography            0.000000
drama                1.113337
enlightening         1.113337
exciting             0.000000
horror               0.835003
inspiring            0.000000
sad                  1.113337
scary                0.835003
thrilling            0.835003
triumphant           0.000000
normalized_rating    2.000000
dtype: float64}
Recommendations for user1:
    title    author   category  rating               emotion  num_r

# If a user has no history (user_profile is None) the recommendation is based on the desired emotion combined with popular books (those with high ratings and more reviews). 
# For users with a history, the recommendations take into account their profile and the desired emotion, ensuring that books with fewer than five reviews do not consider emotions.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import linear_kernel

# Sample dataset with list of emotions, author, category, and number of reviews
books_data = [
    {"title": "Book A", "reviews": "Great book with exciting adventures.", "rating": 4, "emotion": ["exciting", "adventurous"], "author": "Author 1", "category": "Adventure", "num_reviews": 10},
    {"title": "Book B", "reviews": "A sad but enlightening story.", "rating": 5, "emotion": ["sad", "enlightening"], "author": "Author 2", "category": "Drama", "num_reviews": 3},
    {"title": "Book C", "reviews": "An inspiring tale of triumph.", "rating": 4, "emotion": ["inspiring", "triumphant"], "author": "Author 1", "category": "Biography", "num_reviews": 8},
    {"title": "Book D", "reviews": "Scary and thrilling.", "rating": 3, "emotion": ["scary", "thrilling"], "author": "Author 3", "category": "Horror", "num_reviews": 2}
]

# User reading history
user_history = {
    "user1": [{"title": "Book A", "rating": 5}, {"title": "Book C", "rating": 4}],
    "user2": [{"title": "Book B", "rating": 4}, {"title": "Book D", "rating": 3}]
}

# Convert to DataFrame
df_books = pd.DataFrame(books_data)

# Join list of emotions into a single string
df_books['emotion'] = df_books['emotion'].apply(lambda x: ' '.join(x))

# Combine author and category with emotion for feature extraction
df_books['combined_features'] = df_books[['emotion', 'author', 'category']].apply(lambda x: ' '.join(x), axis=1)

# Vectorize the combined features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_books['combined_features'])

# Convert TF-IDF matrix to a DataFrame for better visualization
combined_features = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# Normalize the ratings
scaler = MinMaxScaler()
df_books['normalized_rating'] = scaler.fit_transform(df_books[['rating']])

# Add normalized rating to combined features
combined_features['normalized_rating'] = df_books['normalized_rating']

# Function to create user profiles
def create_user_profile(user_history, df_books, combined_features):
    user_profiles = {}
    
    for user, books in user_history.items():
        user_feature_vector = pd.DataFrame()
        
        for book in books:
            book_title = book['title']
            book_rating = book['rating']
            
            # Get the book features
            book_features = combined_features[df_books['title'] == book_title]
            
            # Scale the book features by the user's rating of the book
            book_features = book_features * book_rating
            
            # Append to user feature vector
            user_feature_vector = pd.concat([user_feature_vector, book_features], axis=0)
        
        # Average the feature vectors to create the user profile
        user_profiles[user] = user_feature_vector.mean(axis=0)
    
    return user_profiles

# Create user profiles
user_profiles = create_user_profile(user_history, df_books, combined_features)
print("User Profiles:")
print(user_profiles)

# Function to recommend books for a user based on their profile and desired emotion
def recommend_books_for_user(user_profile, combined_features, df_books, desired_emotion, top_n=5):
    if user_profile is None:
        # No user profile, recommend based on desired emotion, popular books
        desired_combined_features = ' '.join([desired_emotion, '', ''])
        
        # Vectorize the desired features
        desired_vector = tfidf.transform([desired_combined_features]).toarray()
        
        # Add zero for the normalized rating to match the dimension
        desired_vector = pd.DataFrame(desired_vector, columns=tfidf.get_feature_names_out())
        desired_vector['normalized_rating'] = 0

        # Calculate similarity between desired features and book features
        emotion_similarities = combined_features.apply(lambda x: linear_kernel([x], desired_vector)[0][0], axis=1)
        
        # Add emotion similarity scores to the dataframe
        df_books['emotion_similarity'] = emotion_similarities
        
        # Filter and sort by emotion similarity and rating
        recommended_books = df_books.sort_values(by=['emotion_similarity', 'rating'], ascending=[False, False])
    else:
        # Combine desired emotion with empty author and category for feature extraction
        desired_combined_features = ' '.join([desired_emotion, '', ''])
        
        # Vectorize the desired features
        desired_vector = tfidf.transform([desired_combined_features]).toarray()
        
        # Add zero for the normalized rating to match the dimension
        desired_vector = pd.DataFrame(desired_vector, columns=tfidf.get_feature_names_out())
        desired_vector['normalized_rating'] = 0
        
        # Calculate similarity between user profile and book features
        similarities = combined_features.apply(lambda x: linear_kernel([x], [user_profile])[0][0], axis=1)
        
        # Add similarity scores to the dataframe
        df_books['similarity'] = similarities
        
        # If the number of reviews is less than 5, set emotion similarity to 0
        df_books['emotion_similarity'] = df_books.apply( 
            lambda row: 0 if row['num_reviews'] < 5 else linear_kernel([combined_features.iloc[row.name]], desired_vector)[0][0], axis=1
        )
        # Combine similarity scores with a weight factor (e.g., 0.5 for each)
        df_books['combined_similarity'] = 0.5 * df_books['similarity'] + 0.5 * df_books['emotion_similarity']
        # Filter and sort by combined similarity and rating
        recommended_books = df_books.sort_values(by=['combined_similarity', 'rating'], ascending=[False, False])
        
    return recommended_books[['title', 'author', 'category', 'rating', 'emotion', 'num_reviews']].head(top_n)

# Example usage for user1 with a desired emotion of "exciting adventurous"
user1_profile = user_profiles.get('user1')
user1_recommendations = recommend_books_for_user(user1_profile, combined_features, df_books, "exciting adventurous")
print("Recommendations for user1:")
print(user1_recommendations)

# Example usage for a new user with no history and a desired emotion of "inspiring"
new_user_recommendations = recommend_books_for_user(None, combined_features, df_books, "inspiring triumphant")
print("Recommendations for a new user:")
print(new_user_recommendations)


User Profiles:
{'user1': adventure            1.382013
adventurous          1.382013
author               1.298145
biography            1.105611
drama                0.000000
enlightening         0.000000
exciting             1.382013
horror               0.000000
inspiring            1.105611
sad                  0.000000
scary                0.000000
thrilling            0.000000
triumphant           1.105611
normalized_rating    2.250000
dtype: float64, 'user2': adventure            0.000000
adventurous          0.000000
author               1.009669
biography            0.000000
drama                1.105611
enlightening         1.105611
exciting             0.000000
horror               0.829208
inspiring            0.000000
sad                  1.105611
scary                0.829208
thrilling            0.829208
triumphant           0.000000
normalized_rating    2.000000
dtype: float64}
Recommendations for user1:
    title    author   category  rating               emotion  num_r